# JanataHack: Computer Vision Hackathon
Emergency vs Non-Emergency Vehicle Classification

In [1]:
import pandas as pd
import numpy as np
import pickle
import PIL
import cv2 ,pickle
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
test_data = pd.read_csv("/content/drive/My Drive/Online competition/Analytics vidhya/test_vc2kHdQ.csv") #train csv
train_data = pd.read_csv("/content/drive/My Drive/Online competition/Analytics vidhya/train.csv") #test csv

In [4]:
# Resizing of images in dim (224,224) and converted into array
def array(data_):
  data = []
  for i in data_:
      path = "/content/drive/My Drive/Colab Notebooks/analytics_vidhhya/images/"+i
      img_data = cv2.imread(path)
      img_data = cv2.resize(img_data, (224, 224), interpolation=cv2.INTER_NEAREST)
      data.append(np.array(img_data))
  return(data)

train=array(train_data["image_names"])
test=array(test_data["image_names"])

In [5]:
pickle.dump(train,open("/content/drive/My Drive/Online competition/Analytics vidhya/train.npy","wb"))
pickle.dump(test, open("/content/drive/My Drive/Online competition/Analytics vidhya/test.npy","wb"))

In [6]:
test_img = pickle.load(open("/content/drive/My Drive/Online competition/Analytics vidhya/test.npy","rb"))
train_img = pickle.load(open("/content/drive/My Drive/Online competition/Analytics vidhya/train.npy","rb"))
train_img = np.array(train_img)
test_img = np.array(test_img)

In [7]:
# LabelEncoding and using one hot encoding representation of target variable
encoder = LabelEncoder()
encoder.fit(train_data["emergency_or_not"])
encoded_Y = encoder.transform(train_data["emergency_or_not"])
dummy_y = np_utils.to_categorical(encoded_Y)

# Approach-1

In [ ]:
# Essential libraries for model building
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.wrappers.scikit_learn import KerasClassifier

In [8]:
# Creating a Sequential model
model= Sequential()
model.add(Conv2D(kernel_size=(3,3), filters=3, activation='relu', input_shape=train_img[0].shape))
model.add(Conv2D(filters=3,kernel_size = (3,3),activation='relu'))
model.add(MaxPool2D(3,3))
model.add(Conv2D(filters=3,kernel_size = (3,3),activation='relu'))
model.add(MaxPool2D(3,3))
model.add(Conv2D(filters=3,kernel_size = (3,3),activation='relu'))

model.add(Flatten())

model.add(Dense(20,activation='relu'))
model.add(Dense(15,activation='relu'))
model.add(Dense(2,activation = 'softmax'))
    
model.compile(
              loss='binary_crossentropy', 
              metrics=['acc'],
              optimizer='adam'
             )
model.summary()

In [10]:
# fitting model on training data
history = model.fit(train_img, dummy_y, epochs=5, batch_size=32)

In [11]:
# Predicting target variale on test data
pred = model1.predict(test_img)
pred = encoder.inverse_transform(pred)
result = pd.DataFrame(pred, test_data["image_names"], columns=["emergency_or_not"])
result.to_csv("sample.csv")

# Approach-2
ResNet50 Pre-trained ensemble model

In [12]:
# Essential libraries for model building
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, AveragePooling2D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False, 
    input_shape=(224, 224, 3), 
    pooling='avg'
)

94773248/94765736 [==============================] - 2s 0us/step


In [14]:
base_model.trainable = False
model = Sequential([
  base_model,
  Dense(256, activation="relu"),
  Dropout(0.2),
  Dense(2, activation='sigmoid')
])
sgd = SGD(momentum=0.01, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.summary()

In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('/content/drive/My Drive/Online competition/Analytics vidhya/best_model.h5',
                             monitor='accuracy',
                             verbose=1, save_best_only= True, 
                             mode='auto')

Data Augmentation for better training

In [20]:
batch_size =4
epochs = 5
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,   
        rotation_range=20,
	      width_shift_range=0.1,
	      height_shift_range=0.3,
        brightness_range=[0.2,1.0],
	      horizontal_flip=True,
	      fill_mode="nearest")


training_generator = datagen.flow(train_img, dummy_y,
                                  batch_size=batch_size)

history = model.fit_generator(
         training_generator,
         steps_per_epoch= training_generator.n//training_generator.batch_size,
         callbacks=[checkpoint],
         epochs= epochs)

In [21]:
# Predicting target variable of test data
from tensorflow import keras
model = keras.models.load_model("/content/drive/My Drive/Colab Notebooks/analytics_vidhhya/best_model.h5")
pred = model.predict_classes(test_img)
pred = encoder.inverse_transform(pred)
result = pd.DataFrame(pred, test_data["image_names"], columns=["emergency_or_not"])
result.to_csv("sample.csv")

# Approach-3 
ResNet152 Pre-trained ensemble model

In [22]:

from tensorflow.keras.applications import ResNet152
from keras.applications import VGG16

In [25]:
base_model = ResNet152(
    weights='imagenet',
    include_top=False, 
    input_shape=(224, 224, 3), 
    pooling='avg'
)

In [26]:
base_model.trainable = False

model = Sequential([
  base_model,
  Dense(256, activation="relu"),
  Dropout(0.2),
  Dense(2, activation='sigmoid')
])
sgd = SGD(momentum=0.01, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.summary()

In [27]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('/content/drive/My Drive/Online competition/Analytics vidhya/best_model152.h5',
                             monitor='accuracy',
                             verbose=1, save_best_only= True, 
                             mode='auto')

In [28]:
batch = 5
epochs = 5
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,   
        rotation_range=25,
	      width_shift_range=0.1,
	      height_shift_range=0.3,
        brightness_range=[0.2,1.0],
	      horizontal_flip=True,
	      fill_mode="nearest")

training_generator = datagen.flow(train_img, dummy_y,
                                  batch_size=batch)

history = model.fit_generator(
         training_generator,
         steps_per_epoch= training_generator.n//training_generator.batch_size,
         callbacks=[checkpoint],
         epochs= epochs)

In [29]:
# Predicting target variable of test data
from tensorflow import keras
model = keras.models.load_model("/content/drive/My Drive/Colab Notebooks/analytics_vidhhya/best_model152.h5")

pred = model.predict_classes(test_img)
pred = encoder.inverse_transform(pred)
result = pd.DataFrame(pred, test_data["image_names"], columns=["emergency_or_not"])
result.to_csv("sample.csv")